Going to run through the same process of getting athletes for performances before Olympics and filter performances by date.

Walking through the whole process again:
1. Get all athlete performances >1100 points for 800m -> Marathon
2. Get all outdoor performances and filter out indoor performances, leading to DF1
3. Get list of female athletes who competed in the Olympics through Selenium, leading to DF2
4. Process names and combine DF1 and DF2, ensuring that names are the same.
5. Compute predicted form based on fastest outdoor times ran prior to olympics, with exponential loss decay (and if there are any other models?)
6. Grab Olympic results (finals)
7. Grab IAAF rankings (pre Oly)
8. Compute squared error between predicted rankings (both IAAF rankings pre oly and exponential loss decay)
9. Compute mean squared error and whether volatility was high for the same events (800m and marathon)

Importing statements (to run regardless of progress)


In [1]:
import pandas as pd
!pip install bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
import math


In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


## Step 1 - Get all athlete performances > 1100 points

In [3]:
!pip install bs4
from bs4 import BeautifulSoup


In [4]:
import requests
import pandas as pd


In [5]:
def collect_urls(base_url, start_page, end_page):
  urls = []
  for page in range(start_page, end_page + 1):
        url = base_url.replace("&page=1", f"&page={page}")
        urls.append(url)
  return urls

In [6]:
def get_full_event_results(base_url, start_page, end_page, discipline):
    urls = collect_urls(base_url, start_page, end_page)
    results = []
    for url in urls:
        response = requests.get(url)
        data = response.text
        soup = BeautifulSoup(data, 'html.parser')
        table = soup.find('table')
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            if len(cols) > 1:
                mark = cols[1].text.strip()
                competitor = cols[3].text.strip()
                nationality = cols[5].text.strip()
                location = cols[-3].text.strip()
                date = cols[-2].text.strip()
                result_score = cols[-1].text.strip()

                results.append({
                    'Mark': mark,
                    'Competitor': competitor,
                    'Nat': nationality,
                    'Location': location,
                    'Date': date, #used for form tracking later
                    'Results Score': result_score
                })
    df = pd.DataFrame(results)
    df.insert(4, 'Discipline', discipline)
    return df


In [7]:
base_url_800m = "https://worldathletics.org/records/toplists/middlelong/800-metres/all/women/senior/2024?regionType=world&timing=electronic&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229512&ageCategory=senior"

In [8]:
df_800m_2024 = get_full_event_results(base_url_800m, 1, 14, '800m')
df_800m_2024

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:54.61,Keely HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1261
1,1:55.61,Jemma REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1242
2,1:55.78,Keely HODGKINSON,GBR,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1239
3,1:56.28,Georgia BELL,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1230
4,1:56.71,Mary MORAA,KEN,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1222
...,...,...,...,...,...,...,...
1395,2:03.44,Serena TROIANI,ITA,"Centro di Atletica, Busto Arsizio (ITA)",800m,10 JUL 2024,1101
1396,2:03.44,Mallory LINDAMAN,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",800m,20 JUL 2024,1101
1397,2:03.44,Sarah MORAA,KEN,"Estadio Atlético de la VIDENA, Lima (PER)",800m,29 AUG 2024,1101
1398,2:03.45,Lorena MARTÍN,ESP,"Pista Coberta de Catalunya, Sabadell (ESP) (i)",800m,27 JAN 2024,1129


In [9]:
base_url_1500m = "https://worldathletics.org/records/toplists/middlelong/1500-metres/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229513&ageCategory=senior"

In [10]:
df_1500m_2024_w = get_full_event_results(base_url_1500m, 1, 12, '1500m')
df_1500m_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,3:49.04,Faith KIPYEGON,KEN,"Stade Charléty, Paris (FRA)",1500m,07 JUL 2024,1295
1,3:50.30,Gudaf TSEGAY,ETH,"Egret Stadium, Xiamen (CHN)",1500m,20 APR 2024,1285
2,3:50.83,Jessica HULL,AUS,"Stade Charléty, Paris (FRA)",1500m,07 JUL 2024,1280
3,3:51.29,Faith KIPYEGON,KEN,"Stade de France, Paris (FRA)",1500m,10 AUG 2024,1277
4,3:52.56,Jessica HULL,AUS,"Stade de France, Paris (FRA)",1500m,10 AUG 2024,1266
...,...,...,...,...,...,...,...
1195,4:13.51,Eleanor FULTON,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",1500m,11 MAY 2024,1099
1196,4:13.51,Flomena ASEKOL,KEN,"Meredith College, Raleigh, NC (USA)",1500m,02 AUG 2024,1099
1197,4:13.52,Helen SCHLACHTENHAUFEN,USA,"Drake Stadium, Los Angeles, CA (USA)",1500m,18 MAY 2024,1099
1198,4:13.52,Delia SCLABAS,SUI,"Friidrettsstadion, Jessheim (NOR)",1500m,24 MAY 2024,1099


In [11]:
base_url_3000m_sc = "https://worldathletics.org/records/toplists/middlelong/3000-metres-steeplechase/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229524&ageCategory=senior"

In [12]:
df_3000m_sc_2024_w = get_full_event_results(base_url_3000m_sc, 1, 6, '3000m Steeplechase')
df_3000m_sc_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,8:44.39,Winfred YAVI,BRN,"Stadio Olimpico, Roma (ITA)",3000m Steeplechase,30 AUG 2024,1285
1,8:48.03,Peruth CHEMUTAI,UGA,"Stadio Olimpico, Roma (ITA)",3000m Steeplechase,30 AUG 2024,1275
2,8:52.76,Winfred YAVI,BRN,"Stade de France, Paris (FRA)",3000m Steeplechase,06 AUG 2024,1263
3,8:53.34,Peruth CHEMUTAI,UGA,"Stade de France, Paris (FRA)",3000m Steeplechase,06 AUG 2024,1261
4,8:55.09,Peruth CHEMUTAI,UGA,"Hayward Field, Eugene, OR (USA)",3000m Steeplechase,25 MAY 2024,1257
...,...,...,...,...,...,...,...
595,10:00.17,Poppy TANK,GBR,"Hilmer Lodge Stadium, Walnut, CA (USA)",3000m Steeplechase,18 APR 2024,1095
596,10:00.32,Julia KORALEWSKA,POL,"Estadio Iberoamericano, Huelva (ESP)",3000m Steeplechase,30 APR 2024,1094
597,10:00.32,Marta SERRANO,ESP,"Estadio Iberoamericano, Huelva (ESP)",3000m Steeplechase,30 APR 2024,1094
598,10:00.32,Tatiane Raquel DA SILVA,BRA,"Centro de Treinamento Paralímpico Brasileiro, ...",3000m Steeplechase,27 JUN 2024,1094


In [13]:
base_url_5000m = "https://worldathletics.org/records/toplists/middlelong/5000-metres/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229514&ageCategory=senior"

In [14]:
df_5000m_2024_w = get_full_event_results(base_url_5000m, 1, 6, '5000m')
df_5000m_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,14:18.76,Tsigie GEBRESELAMA,ETH,"Hayward Field, Eugene, OR (USA)",5000m,25 MAY 2024,1244
1,14:18.92,Ejgayehu TAYE,ETH,"Hayward Field, Eugene, OR (USA)",5000m,25 MAY 2024,1244
2,14:20.61,Freweyni HAILU,ETH,"Hayward Field, Eugene, OR (USA)",5000m,25 MAY 2024,1241
3,14:22.76,Aynadis MEBRATU,ETH,"Hayward Field, Eugene, OR (USA)",5000m,25 MAY 2024,1236
4,14:23.71,Birke HAYLOM,ETH,"Hayward Field, Eugene, OR (USA)",5000m,25 MAY 2024,1234
...,...,...,...,...,...,...,...
595,15:36.46,Yuna TAKAHASHI,JPN,"Athletic Track, Abashiri (JPN)",5000m,10 JUL 2024,1093
596,15:36.48,Aliandrea UPSHAW,USA,"Dempsey Indoor, Seattle, WA (USA)",5000m,09 FEB 2024,1093
597,15:36.49,Mica RIVERA,PER,"Lincoln HS, Portland, OR (USA)",5000m,09 JUN 2024,1093
598,15:36.53,Jana VAN LENT,BEL,"Boudewijnstadion, Bruxelles (BEL)",5000m,29 JUN 2024,1093


In [15]:
base_url_10k = "https://worldathletics.org/records/toplists/middlelong/10000-metres/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229521&ageCategory=senior"

In [16]:
df_10k_2024_w = get_full_event_results(base_url_10k, 1, 3, '10000m')
df_10k_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,28:54.14,Beatrice CHEBET,KEN,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1309
1,29:05.92,Gudaf TSEGAY,ETH,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1298
2,29:26.89,Lilian Kasait RENGERUK,KEN,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1278
3,29:27.59,Margaret Chelimo KIPKEMBOI,KEN,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1278
4,29:47.71,Fotyen TESFAY,ETH,"Estadio Enrique Lopez Cuenca, Nerja (ESP)",10000m,14 JUN 2024,1259
...,...,...,...,...,...,...,...
295,33:06.73,Jennifer SANDOVAL-GUZMAN,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
296,33:06.80,Katrina SPRATFORD-STERLING,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
297,33:06.99,Mariya NODA,JPN,"Athletic Track, Abashiri (JPN)",10000m,10 JUL 2024,1081
298,33:07.01,Katrina SPRATFORD-STERLING,USA,"JSerra Catholic HS, San Juan Capistrano, CA (USA)",10000m,16 MAR 2024,1081


In [17]:
base_url_hm = "https://worldathletics.org/records/toplists/road-running/half-marathon/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229541&ageCategory=senior"

In [18]:
df_hm_2024_w = get_full_event_results(base_url_hm, 1, 3, 'Half Marathon')
df_hm_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:04:29,Joyciline JEPKOSGEI,KEN,Barcelona (ESP),Half Marathon,11 FEB 2024,1246
1,1:04:37,Sutume Asefa KEBEDE,ETH,"Houston, TX (USA)",Half Marathon,14 JAN 2024,1243
2,1:04:40,Senbere TEFERI,ETH,Barcelona (ESP),Half Marathon,11 FEB 2024,1242
3,1:05:14,Tsigie GEBRESELAMA,ETH,Ras Al Khaimah (UAE),Half Marathon,24 FEB 2024,1230
4,1:05:44,Ababel YESHANEH,ETH,Ras Al Khaimah (UAE),Half Marathon,24 FEB 2024,1219
...,...,...,...,...,...,...,...
295,1:11:46,Glorius JEPKIRUI,KEN,Klagenfurt (AUT),Half Marathon,25 AUG 2024,1094
296,1:11:47,Jackline CHELAL,KEN,Madrid (ESP),Half Marathon,07 APR 2024,1093
297,1:11:47,Maryna NEMCHENKO,UKR,Poznan (POL),Half Marathon,14 APR 2024,1093
298,1:11:48,Angie ORJUELA,COL,"Houston, TX (USA)",Half Marathon,14 JAN 2024,1093


In [19]:
base_url_m = "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2024?regionType=world&page=1&bestResultsOnly=false&maxResultsByCountry=all&eventId=10229534&ageCategory=senior"


In [20]:
df_m_2024_w = get_full_event_results(base_url_m, 1, 5, 'Marathon')
df_m_2024_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,2:15:55,Sutume Asefa KEBEDE,ETH,Tokyo (JPN),Marathon,03 MAR 2024,1276
1,2:16:07,Tigist KETEMA,ETH,Dubai (UAE),Marathon,07 JAN 2024,1274
2,2:16:14,Rosemary WANJIRU,KEN,Tokyo (JPN),Marathon,03 MAR 2024,1272
3,2:16:16,Peres JEPCHIRCHIR,KEN,London (GBR),Marathon,21 APR 2024,1272
4,2:16:23,Tigst ASSEFA,ETH,London (GBR),Marathon,21 APR 2024,1271
...,...,...,...,...,...,...,...
495,2:34:58,Amber ZIMMERMAN,USA,"Orlando, FL (USA)",Marathon,03 FEB 2024,1084
496,2:34:58,Alemitu HAROYE,ETH,Zheng-Kai (CHN),Marathon,31 MAR 2024,1084
497,2:34:59,Genet ROBI,ETH,Shijiazhuang (CHN),Marathon,31 MAR 2024,1084
498,2:35:01,Kasumi NISHIHARA,JPN,Tokyo (JPN),Marathon,03 MAR 2024,1084


In [21]:
combined_df_w_road = pd.concat([df_m_2024_w, df_hm_2024_w], ignore_index=True)
combined_df_w_road

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,2:15:55,Sutume Asefa KEBEDE,ETH,Tokyo (JPN),Marathon,03 MAR 2024,1276
1,2:16:07,Tigist KETEMA,ETH,Dubai (UAE),Marathon,07 JAN 2024,1274
2,2:16:14,Rosemary WANJIRU,KEN,Tokyo (JPN),Marathon,03 MAR 2024,1272
3,2:16:16,Peres JEPCHIRCHIR,KEN,London (GBR),Marathon,21 APR 2024,1272
4,2:16:23,Tigst ASSEFA,ETH,London (GBR),Marathon,21 APR 2024,1271
...,...,...,...,...,...,...,...
795,1:11:46,Glorius JEPKIRUI,KEN,Klagenfurt (AUT),Half Marathon,25 AUG 2024,1094
796,1:11:47,Jackline CHELAL,KEN,Madrid (ESP),Half Marathon,07 APR 2024,1093
797,1:11:47,Maryna NEMCHENKO,UKR,Poznan (POL),Half Marathon,14 APR 2024,1093
798,1:11:48,Angie ORJUELA,COL,"Houston, TX (USA)",Half Marathon,14 JAN 2024,1093


In [22]:
combined_df_track_w = pd.concat([df_800m_2024, df_1500m_2024_w, df_3000m_sc_2024_w, df_5000m_2024_w, df_10k_2024_w], ignore_index=True)
combined_df_track_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:54.61,Keely HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1261
1,1:55.61,Jemma REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1242
2,1:55.78,Keely HODGKINSON,GBR,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1239
3,1:56.28,Georgia BELL,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1230
4,1:56.71,Mary MORAA,KEN,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1222
...,...,...,...,...,...,...,...
4095,33:06.73,Jennifer SANDOVAL-GUZMAN,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4096,33:06.80,Katrina SPRATFORD-STERLING,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4097,33:06.99,Mariya NODA,JPN,"Athletic Track, Abashiri (JPN)",10000m,10 JUL 2024,1081
4098,33:07.01,Katrina SPRATFORD-STERLING,USA,"JSerra Catholic HS, San Juan Capistrano, CA (USA)",10000m,16 MAR 2024,1081


In [23]:
combined_df_track_w['Competitor'] = combined_df_track_w['Competitor'].str.upper()
combined_df_track_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:54.61,KEELY HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1261
1,1:55.61,JEMMA REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1242
2,1:55.78,KEELY HODGKINSON,GBR,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1239
3,1:56.28,GEORGIA BELL,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1230
4,1:56.71,MARY MORAA,KEN,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1222
...,...,...,...,...,...,...,...
4095,33:06.73,JENNIFER SANDOVAL-GUZMAN,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4096,33:06.80,KATRINA SPRATFORD-STERLING,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4097,33:06.99,MARIYA NODA,JPN,"Athletic Track, Abashiri (JPN)",10000m,10 JUL 2024,1081
4098,33:07.01,KATRINA SPRATFORD-STERLING,USA,"JSerra Catholic HS, San Juan Capistrano, CA (USA)",10000m,16 MAR 2024,1081


In [24]:
combined_df_track_w.to_csv('combined_df_track_w.csv', index=False)

In [25]:
combined_distance_w = pd.concat([combined_df_track_w, combined_df_w_road], ignore_index=True)
combined_distance_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:54.61,KEELY HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1261
1,1:55.61,JEMMA REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1242
2,1:55.78,KEELY HODGKINSON,GBR,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1239
3,1:56.28,GEORGIA BELL,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1230
4,1:56.71,MARY MORAA,KEN,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1222
...,...,...,...,...,...,...,...
4895,1:11:46,Glorius JEPKIRUI,KEN,Klagenfurt (AUT),Half Marathon,25 AUG 2024,1094
4896,1:11:47,Jackline CHELAL,KEN,Madrid (ESP),Half Marathon,07 APR 2024,1093
4897,1:11:47,Maryna NEMCHENKO,UKR,Poznan (POL),Half Marathon,14 APR 2024,1093
4898,1:11:48,Angie ORJUELA,COL,"Houston, TX (USA)",Half Marathon,14 JAN 2024,1093


In [26]:
combined_distance_w.to_csv('combined_distance_w.csv', index=False)

## Step 2

In [27]:
def remove_all_indoor(df):
    new_df = df[~df['Location'].str.contains(r'\(i\)', case=False)]
    return new_df
outdoor_df_w = remove_all_indoor(combined_distance_w)
outdoor_df_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,1:54.61,KEELY HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1261
1,1:55.61,JEMMA REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1242
2,1:55.78,KEELY HODGKINSON,GBR,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1239
3,1:56.28,GEORGIA BELL,GBR,"Olympic Stadium, London (GBR)",800m,20 JUL 2024,1230
4,1:56.71,MARY MORAA,KEN,"Hayward Field, Eugene, OR (USA)",800m,25 MAY 2024,1222
...,...,...,...,...,...,...,...
4895,1:11:46,Glorius JEPKIRUI,KEN,Klagenfurt (AUT),Half Marathon,25 AUG 2024,1094
4896,1:11:47,Jackline CHELAL,KEN,Madrid (ESP),Half Marathon,07 APR 2024,1093
4897,1:11:47,Maryna NEMCHENKO,UKR,Poznan (POL),Half Marathon,14 APR 2024,1093
4898,1:11:48,Angie ORJUELA,COL,"Houston, TX (USA)",Half Marathon,14 JAN 2024,1093


In [28]:
outdoor_df_w["Results Score"] = outdoor_df_w["Results Score"].astype(int)
outdoor_df_w["Results Score"].dtype

<ipython-input-28-7a83e1d5d596>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outdoor_df_w["Results Score"] = outdoor_df_w["Results Score"].astype(int)


dtype('int64')

In [29]:
outdoor_df_sorted_w = outdoor_df_w.sort_values(by="Results Score", ascending=False)
outdoor_df_sorted_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
3800,28:54.14,BEATRICE CHEBET,KEN,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1309
3801,29:05.92,GUDAF TSEGAY,ETH,"Hayward Field, Eugene, OR (USA)",10000m,25 MAY 2024,1298
1400,3:49.04,FAITH KIPYEGON,KEN,"Stade Charléty, Paris (FRA)",1500m,07 JUL 2024,1295
2600,8:44.39,WINFRED YAVI,BRN,"Stadio Olimpico, Roma (ITA)",3000m Steeplechase,30 AUG 2024,1285
1401,3:50.30,GUDAF TSEGAY,ETH,"Egret Stadium, Xiamen (CHN)",1500m,20 APR 2024,1285
...,...,...,...,...,...,...,...
4095,33:06.73,JENNIFER SANDOVAL-GUZMAN,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4096,33:06.80,KATRINA SPRATFORD-STERLING,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",10000m,11 MAY 2024,1081
4097,33:06.99,MARIYA NODA,JPN,"Athletic Track, Abashiri (JPN)",10000m,10 JUL 2024,1081
4098,33:07.01,KATRINA SPRATFORD-STERLING,USA,"JSerra Catholic HS, San Juan Capistrano, CA (USA)",10000m,16 MAR 2024,1081


In [30]:
outdoor_df_sorted_w['Date'] = pd.to_datetime(outdoor_df_sorted_w['Date'], format = '%d %b %Y')
filtered_df = outdoor_df_sorted_w[outdoor_df_sorted_w['Date']<'2024-08-01'] # we want any performances before the Olympics so
filtered_df.sort_values(by=['Competitor', 'Discipline', 'Date'], ascending=[True, True, False], inplace=True)

# Drop duplicates, keeping the last (most recent) entry for each competitor
outdoor_df_sorted_by_latest_w = filtered_df.drop_duplicates(subset=['Competitor','Discipline'], keep='first')

outdoor_df_sorted_by_latest_w

<ipython-input-30-6ddbc5a7bfac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by=['Competitor', 'Discipline', 'Date'], ascending=[True, True, False], inplace=True)


,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
3004,9:48.26,. PRITI,IND,"Dean A. Hayes Track and Soccer Stadium, Murfre...",3000m Steeplechase,2024-05-31,1124
4058,32:55.91,. SEEMA,IND,"Mt. Hood Community College, Gresham, OR (USA)",10000m,2024-06-08,1090
3680,15:30.70,. SEEMA,IND,"Drake Stadium, Los Angeles, CA (USA)",5000m,2024-05-17,1104
1227,2:03.08,AALIYAH MILLER,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",800m,2024-07-20,1108
980,2:02.50,AALIYAH MOORE,GUY,"John McDonnell Field, Fayetteville, AR (USA)",800m,2024-05-25,1118
...,...,...,...,...,...,...,...
4398,2:30:02,Zinash DEBEBE,ETH,Praha (CZE),Marathon,2024-05-05,1132
1647,4:03.90,ÁGUEDA MARQUÉS,ESP,"Estadi Olímpic Camilo Cano, La Nucia (ESP)",1500m,2024-06-30,1174
970,2:02.47,ÁGUEDA MARQUÉS,ESP,"Stade Jean Jacoby, Schifflange (LUX)",800m,2024-07-21,1118
2085,4:09.34,ŞILAN AYYILDIZ,TUR,"Drake Stadium, Des Moines, IA (USA)",1500m,2024-04-27,1132


In [31]:
outdoor_df_sorted_by_latest_w.to_csv('outdoor_df_sorted_by_latest_w.csv', index=False)

## Step 3

### Selenium Config

In [32]:
!pip install selenium

In [33]:
!apt-get update
!apt-get install chromium chromium-driver

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [962 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [

In [34]:
from selenium import webdriver

In [35]:
!apt-get install chromium-driver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://ar

In [36]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver



In [37]:
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [38]:
urls = [
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=800",
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=1500",
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=3KSC",
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=5000",
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=10K",
    "https://worldathletics.org/competitions/olympic-games/paris24/athletes?competitionGroup=olympic-games&urlSlug=paris24&sexCode=W&disciplineCode=MAR"
]
events = [
    "800m",
    "1500m",
    "3000m SC",
    "5000m",
    "10000m",
    "Marathon",
]


In [39]:
def fetch_athlete_data(driver, urls, events):
    all_athletes = []

    for url, event in zip(urls, events):
        driver.get(url)

        # Wait for the table to be present on the page
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.styles_table__oNcJD"))
            )
        except Exception as e:
            print(f"An error occurred while waiting for the table for event {event}:", e)
            continue

        # Optional: Pause to ensure all JavaScript content is loaded
        time.sleep(5)  # Adjust the sleep time as necessary

        # Get the page source after rendering
        raw_html = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(raw_html, 'html.parser')

        # Find the table
        table = soup.find('table', class_='styles_table__oNcJD')
        if table:
            for row in table.find_all('tr')[1:]:  # Skip the header row
                cols = row.find_all('td')
                if len(cols) > 1:
                    athlete = cols[1].text.strip()
                    nationality = cols[3].text.strip()
                    birthdate = cols[4].text.strip()
                    all_athletes.append({
                        'Athlete': athlete,
                        'Nat': nationality,
                        'Birthdate': birthdate,
                        'Event': event
                    })
        else:
            print(f"Table not found for event {event}")

    # Convert to DataFrame
    df = pd.DataFrame(all_athletes)
    return df


In [40]:
driver = web_driver()

In [41]:
df_oly_w = fetch_athlete_data(driver, urls, events)
df_oly_w


,Athlete,Nat,Birthdate,Event
0,"AKINS, Nia",USA,07 Jul 1998,800m
1,"AL ROUMI, Amal",KUW,22 Aug 1992,800m
2,"ALEMU, Habitam",ETH,09 Jul 1997,800m
3,"ALMANZA, Rose Mary",CUB,13 Jul 1992,800m
4,"ALMASRI, Layla",PLE,26 Jun 1999,800m
...,...,...,...,...
284,"WIKSTRÖM, Carolina",SWE,04 Sep 1993,Marathon
285,"WOLDU, Mekdes",FRA,20 Oct 1992,Marathon
286,"XIA, Yuyu",CHN,01 Mar 1998,Marathon
287,"YAREMCHUK, Sofiia",ITA,03 Jun 1994,Marathon


In [42]:
df_oly_w.to_csv('df_w_distance_without_formatting.csv', index=False)

In [43]:
df_oly_w = pd.read_csv('df_w_distance_without_formatting.csv')
df_oly_w

,Athlete,Nat,Birthdate,Event
0,"AKINS, Nia",USA,07 Jul 1998,800m
1,"AL ROUMI, Amal",KUW,22 Aug 1992,800m
2,"ALEMU, Habitam",ETH,09 Jul 1997,800m
3,"ALMANZA, Rose Mary",CUB,13 Jul 1992,800m
4,"ALMASRI, Layla",PLE,26 Jun 1999,800m
...,...,...,...,...
284,"WIKSTRÖM, Carolina",SWE,04 Sep 1993,Marathon
285,"WOLDU, Mekdes",FRA,20 Oct 1992,Marathon
286,"XIA, Yuyu",CHN,01 Mar 1998,Marathon
287,"YAREMCHUK, Sofiia",ITA,03 Jun 1994,Marathon


In [44]:
# prompt: Join the athlete names and swap the names that are after the comma to before the comma and make everything into block letters.

def format_names(df):
  df['Athlete'] = df['Athlete'].str.upper()
  split_names = df['Athlete'].str.split(', ', n=1, expand=True)
  df['Athlete'] = split_names[1].str.cat(split_names[0], sep=' ').fillna(df['Athlete'])
  return df

df_oly_w_formatted = format_names(df_oly_w)
df_oly_w_formatted


,Athlete,Nat,Birthdate,Event
0,NIA AKINS,USA,07 Jul 1998,800m
1,AMAL AL ROUMI,KUW,22 Aug 1992,800m
2,HABITAM ALEMU,ETH,09 Jul 1997,800m
3,ROSE MARY ALMANZA,CUB,13 Jul 1992,800m
4,LAYLA ALMASRI,PLE,26 Jun 1999,800m
...,...,...,...,...
284,CAROLINA WIKSTRÖM,SWE,04 Sep 1993,Marathon
285,MEKDES WOLDU,FRA,20 Oct 1992,Marathon
286,YUYU XIA,CHN,01 Mar 1998,Marathon
287,SOFIIA YAREMCHUK,ITA,03 Jun 1994,Marathon


In [45]:
df_oly_w_formatted['Event'] = df_oly_w_formatted.groupby('Athlete')['Event'].transform(lambda x: ', '.join(sorted(x.unique())))
df_oly_w_formatted = df_oly_w_formatted.drop_duplicates(subset='Athlete')
df_oly_w_formatted

,Athlete,Nat,Birthdate,Event
0,NIA AKINS,USA,07 Jul 1998,800m
1,AMAL AL ROUMI,KUW,22 Aug 1992,800m
2,HABITAM ALEMU,ETH,09 Jul 1997,800m
3,ROSE MARY ALMANZA,CUB,13 Jul 1992,800m
4,LAYLA ALMASRI,PLE,26 Jun 1999,800m
...,...,...,...,...
284,CAROLINA WIKSTRÖM,SWE,04 Sep 1993,Marathon
285,MEKDES WOLDU,FRA,20 Oct 1992,Marathon
286,YUYU XIA,CHN,01 Mar 1998,Marathon
287,SOFIIA YAREMCHUK,ITA,03 Jun 1994,Marathon


In [46]:
df_oly_w_formatted.to_csv('df_oly_w_formatted.csv', index=False)


In [47]:
outdoor_df_sorted_by_latest_w = pd.read_csv('outdoor_df_sorted_by_latest_w.csv')
outdoor_df_sorted_by_latest_w

,Mark,Competitor,Nat,Location,Discipline,Date,Results Score
0,9:48.26,. PRITI,IND,"Dean A. Hayes Track and Soccer Stadium, Murfre...",3000m Steeplechase,2024-05-31,1124
1,32:55.91,. SEEMA,IND,"Mt. Hood Community College, Gresham, OR (USA)",10000m,2024-06-08,1090
2,15:30.70,. SEEMA,IND,"Drake Stadium, Los Angeles, CA (USA)",5000m,2024-05-17,1104
3,2:03.08,AALIYAH MILLER,USA,"Jack Kemp Stadium, Los Angeles, CA (USA)",800m,2024-07-20,1108
4,2:02.50,AALIYAH MOORE,GUY,"John McDonnell Field, Fayetteville, AR (USA)",800m,2024-05-25,1118
...,...,...,...,...,...,...,...
1829,2:30:02,Zinash DEBEBE,ETH,Praha (CZE),Marathon,2024-05-05,1132
1830,4:03.90,ÁGUEDA MARQUÉS,ESP,"Estadi Olímpic Camilo Cano, La Nucia (ESP)",1500m,2024-06-30,1174
1831,2:02.47,ÁGUEDA MARQUÉS,ESP,"Stade Jean Jacoby, Schifflange (LUX)",800m,2024-07-21,1118
1832,4:09.34,ŞILAN AYYILDIZ,TUR,"Drake Stadium, Des Moines, IA (USA)",1500m,2024-04-27,1132


In [48]:
# prompt: make all names in the competitor field in block letters (capital)

outdoor_df_sorted_by_latest_w['Competitor'] = outdoor_df_sorted_by_latest_w['Competitor'].str.upper()


In [49]:
merged_df = pd.merge(outdoor_df_sorted_by_latest_w, df_oly_w_formatted, left_on='Competitor', right_on='Athlete', how='inner')
merged_df.drop(columns=['Athlete'], inplace = True)
merged_df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event
0,4:05.45,ABBEY CALDWELL,AUS,"Zdzisław Krzyszkowiak Stadium, Bydgoszcz (POL)",1500m,2024-06-20,1162,AUS,03 Jul 2001,800m
1,1:59.35,ABBEY CALDWELL,AUS,"Stadion Allmend, Luzern (SUI)",800m,2024-07-16,1174,AUS,03 Jul 2001,800m
2,15:28.04,AGATE CAUNE,LAT,"Stadio Olimpico, Roma (ITA)",5000m,2024-06-07,1109,LAT,07 Aug 2004,5000m
3,3:58.05,AGATHE GUILLEMOT,FRA,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1221,FRA,11 Jul 1999,1500m
4,2:01.05,AGATHE GUILLEMOT,FRA,"Stade des Maradas, Cergy-Pontoise (FRA)",800m,2024-05-12,1144,FRA,11 Jul 1999,1500m
...,...,...,...,...,...,...,...,...,...,...
372,4:12.85,YUME GOTO,JPN,"Denka Big Swan Stadium, Niigata (JPN)",1500m,2024-06-28,1104,JPN,25 Feb 2000,1500m
373,2:25:45,YUYU XIA,CHN,Nagoya (JPN),Marathon,2024-03-10,1175,CHN,01 Mar 1998,Marathon
374,2:26:42,ZHANNA MAMAZHANOVA,KAZ,Rotterdam (NED),Marathon,2024-04-14,1165,KAZ,26 Jan 1994,Marathon
375,4:03.90,ÁGUEDA MARQUÉS,ESP,"Estadi Olímpic Camilo Cano, La Nucia (ESP)",1500m,2024-06-30,1174,ESP,19 Mar 1999,1500m


In [50]:
merged_df.to_csv('merged_df_w_new.csv', index=False)


In [51]:
sorted_df = merged_df.sort_values(by='Results Score', ascending=False)
sorted_df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event
32,28:54.14,BEATRICE CHEBET,KEN,"Hayward Field, Eugene, OR (USA)",10000m,2024-05-25,1309,KEN,05 Mar 2000,"10000m, 5000m"
141,29:05.92,GUDAF TSEGAY,ETH,"Hayward Field, Eugene, OR (USA)",10000m,2024-05-25,1298,ETH,23 Jan 1997,"10000m, 1500m, 5000m"
111,3:49.04,FAITH KIPYEGON,KEN,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1295,KEN,10 Jan 1994,"1500m, 5000m"
142,3:50.30,GUDAF TSEGAY,ETH,"Egret Stadium, Xiamen (CHN)",1500m,2024-04-20,1285,ETH,23 Jan 1997,"10000m, 1500m, 5000m"
165,3:50.83,JESSICA HULL,AUS,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1280,AUS,22 Oct 1996,1500m
...,...,...,...,...,...,...,...,...,...,...
300,15:36.14,RINO GOSHIMA,JPN,"National Stadium, Tokyo (JPN)",5000m,2024-05-19,1094,JPN,29 Oct 1997,10000m
28,1:11:48,ANGIE ORJUELA,COL,"Houston, TX (USA)",Half Marathon,2024-01-14,1093,COL,09 May 1989,Marathon
249,1:11:48,MATEA PARLOV KOŠTRO,CRO,"Stadio Olimpico, Roma (ITA)",Half Marathon,2024-06-09,1093,CRO,02 Jun 1992,Marathon
306,2:34:08,REBECCA CHEPTEGEI,UGA,Doha (QAT),Marathon,2024-02-16,1092,UGA,22 Feb 1991,Marathon


### Calculating Exponential Loss Decay

In [52]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df["Days Since Performance"] = (pd.to_datetime('2024-08-01') - merged_df['Date']).dt.days #to make this more precise the actual event date can be used but can be aggregated to 1 aug for now

In [53]:
lambda_decay = 0.001  # ideally this is changeable

# Calculate the weighted score using exponential decay
merged_df['Weighted Score'] = merged_df['Results Score'] * np.exp(-lambda_decay * merged_df['Days Since Performance'])

In [54]:
sorted_df = merged_df.sort_values(by='Weighted Score', ascending=False)
sorted_df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event,Days Since Performance,Weighted Score
111,3:49.04,FAITH KIPYEGON,KEN,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1295,KEN,10 Jan 1994,"1500m, 5000m",25,1263.026336
165,3:50.83,JESSICA HULL,AUS,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1280,AUS,22 Oct 1996,1500m,25,1248.396687
187,1:54.61,KEELY HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,2024-07-20,1261,GBR,03 Mar 2002,800m,12,1245.958430
164,1:55.61,JEMMA REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,2024-07-20,1242,GBR,06 Mar 1998,800m,12,1227.185067
199,3:53.79,LAURA MUIR,GBR,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1256,GBR,09 May 1993,1500m,25,1224.989250
...,...,...,...,...,...,...,...,...,...,...,...,...
179,1:11:09,JULIA MAYER,AUT,Barcelona (ESP),Half Marathon,2024-02-11,1106,AUT,20 Jan 1993,Marathon,172,931.228966
255,1:10:21,MERITXELL SOLER,ESP,Santa Pola (ESP),Half Marathon,2024-01-21,1122,ESP,20 Jul 1992,Marathon,193,925.068775
306,2:34:08,REBECCA CHEPTEGEI,UGA,Doha (QAT),Marathon,2024-02-16,1092,UGA,22 Feb 1991,Marathon,167,924.049976
63,1:10:46,CAROLINA WIKSTRÖM,SWE,Santa Pola (ESP),Half Marathon,2024-01-21,1114,SWE,04 Sep 1993,Marathon,193,918.472919


In [55]:
sorted_df.to_csv('sorted_df_w_new.csv', index=False)


## Post Olympics Form Analysis

In [56]:
df = pd.read_csv('sorted_df_w_new.csv')
df

,Mark,Competitor,Nat_x,Location,Discipline,Date,Results Score,Nat_y,Birthdate,Event,Days Since Performance,Weighted Score
0,3:49.04,FAITH KIPYEGON,KEN,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1295,KEN,10 Jan 1994,"1500m, 5000m",25,1263.026336
1,3:50.83,JESSICA HULL,AUS,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1280,AUS,22 Oct 1996,1500m,25,1248.396687
2,1:54.61,KEELY HODGKINSON,GBR,"Olympic Stadium, London (GBR)",800m,2024-07-20,1261,GBR,03 Mar 2002,800m,12,1245.958430
3,1:55.61,JEMMA REEKIE,GBR,"Olympic Stadium, London (GBR)",800m,2024-07-20,1242,GBR,06 Mar 1998,800m,12,1227.185067
4,3:53.79,LAURA MUIR,GBR,"Stade Charléty, Paris (FRA)",1500m,2024-07-07,1256,GBR,09 May 1993,1500m,25,1224.989250
...,...,...,...,...,...,...,...,...,...,...,...,...
372,1:11:09,JULIA MAYER,AUT,Barcelona (ESP),Half Marathon,2024-02-11,1106,AUT,20 Jan 1993,Marathon,172,931.228966
373,1:10:21,MERITXELL SOLER,ESP,Santa Pola (ESP),Half Marathon,2024-01-21,1122,ESP,20 Jul 1992,Marathon,193,925.068775
374,2:34:08,REBECCA CHEPTEGEI,UGA,Doha (QAT),Marathon,2024-02-16,1092,UGA,22 Feb 1991,Marathon,167,924.049976
375,1:10:46,CAROLINA WIKSTRÖM,SWE,Santa Pola (ESP),Half Marathon,2024-01-21,1114,SWE,04 Sep 1993,Marathon,193,918.472919


In [57]:
def get_olympic_event_results(url, discipline):
    results = []

    # Fetch the webpage
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')

    # Locate the table containing the results
    table = soup.find('table')

    # Ensure the table exists
    if table:
        # Iterate over each row in the table (skipping the header row)
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            if len(cols) > 1:
                place = cols[0].text.strip()
                nationality = cols[2].text.strip()
                competitor = cols[3].text.strip()
                mark = cols[4].text.strip()


                results.append({
                    'Place': place,
                    'Competitor': competitor,
                    'Nat': nationality,
                    'Mark': mark,
                }) #what about the results score? would probably have to scrape data (again) for post OLY results lmao

    # Convert the results into a DataFrame
    df = pd.DataFrame(results)

    # Insert the discipline column
    df.insert(4, 'Discipline', discipline)

    return df


In [58]:
url = "https://worldathletics.org/competitions/olympic-games/paris24/results/women/1500-metres/final/result"
discipline = "1500m"
df_1500_results = get_olympic_event_results(url, discipline)

df_1500_results

,Place,Competitor,Nat,Mark,Discipline
0,1,Faith KIPYEGON,KEN,3:51.29 OR,1500m
1,2,Jessica HULL,AUS,3:52.56,1500m
2,3,Georgia BELL,GBR,3:52.61 NR,1500m
3,4,Diribe WELTEJI,ETH,3:52.75 PB,1500m
4,5,Laura MUIR,GBR,3:53.37 PB,1500m
5,6,Susan Lokayo EJORE,KEN,3:56.07 PB,1500m
6,7,Nikki HILTZ,USA,3:56.38,1500m
7,8,Elle ST. PIERRE,USA,3:57.52,1500m
8,9,Agathe GUILLEMOT,FRA,3:59.08,1500m
9,10,Klaudia KAZIMIERSKA,POL,4:00.12 PB,1500m


In [59]:
df_1500_results['Competitor'] = df_1500_results['Competitor'].str.upper()
df_1500_results


,Place,Competitor,Nat,Mark,Discipline
0,1,FAITH KIPYEGON,KEN,3:51.29 OR,1500m
1,2,JESSICA HULL,AUS,3:52.56,1500m
2,3,GEORGIA BELL,GBR,3:52.61 NR,1500m
3,4,DIRIBE WELTEJI,ETH,3:52.75 PB,1500m
4,5,LAURA MUIR,GBR,3:53.37 PB,1500m
5,6,SUSAN LOKAYO EJORE,KEN,3:56.07 PB,1500m
6,7,NIKKI HILTZ,USA,3:56.38,1500m
7,8,ELLE ST. PIERRE,USA,3:57.52,1500m
8,9,AGATHE GUILLEMOT,FRA,3:59.08,1500m
9,10,KLAUDIA KAZIMIERSKA,POL,4:00.12 PB,1500m


In [60]:
def find_index_in_larger_df(competitor_name, larger_df):
    try:
        return (larger_df[larger_df['Competitor'] == competitor_name].index[0])+1  # Get the first occurrence
    except IndexError:
        return None  # Return None if the competitor is not found



In [61]:
def get_filtered_event_df(df, event):
    return df[df['Event'].str.contains(event)]


In [62]:
def fetch_athlete_data(driver, urls):
    all_athletes = []

    for url in urls:
        driver.get(url)

        # Wait for the table to be present on the page
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "records-table"))
            )
        except Exception as e:
            print(f"An error occurred while waiting for the table at {url}:", e)
            continue


        time.sleep(5)

        raw_html = driver.page_source

        soup = BeautifulSoup(raw_html, 'html.parser')


        table = soup.find('table', class_='records-table')
        if table:
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                if len(cols) > 1:
                    rank = cols[0].text.strip()
                    athlete = cols[1].text.strip()
                    nationality = cols[2].text.strip()
                    score = cols[3].text.strip()
                    event_list = cols[4].text.strip()

                    all_athletes.append({
                        'Rank': rank,
                        'Athlete': athlete,
                        'Nat': nationality,
                        'Score': score,
                        'Event List': event_list
                    })
        else:
            print(f"Table not found at {url}")

    # Convert to DataFrame
    df = pd.DataFrame(all_athletes)
    return df


# Example usage:
# Assuming driver is a Selenium WebDriver instance that has been properly configured





In [70]:
import numpy as np
import pandas as pd

def complete_analysis(result_url, event, ranking_urls):
    df_results = get_olympic_event_results(result_url, event)
    filtered_event_df = get_filtered_event_df(df, event)
    df_results['Competitor'] = df_results['Competitor'].str.upper()
    df_results['Index in rerank'] = df_results['Competitor'].apply(find_index_in_larger_df, args=(filtered_event_df,))
    driver = web_driver()
    df_rankings = fetch_athlete_data(driver, ranking_urls)
    df_rankings = df_rankings.rename(columns={'Athlete': 'Competitor'})
    df_rankings['Competitor'] = df_rankings['Competitor'].str.upper()
    df_results['Index in IAAF rank'] = df_results['Competitor'].apply(find_index_in_larger_df, args=(df_rankings,))
    df_results['Index in rerank'] = pd.to_numeric(df_results['Index in rerank'], errors='coerce')
    df_results['Index in IAAF rank'] = pd.to_numeric(df_results['Index in IAAF rank'], errors='coerce')
    df_results['Place'] = pd.to_numeric(df_results['Place'], errors='coerce')

    # Calculate differences
    df_results['Diff from Rerank'] = df_results['Index in rerank'] - df_results['Place']
    df_results['Diff from IAAF'] = df_results['Index in IAAF rank'] - df_results['Place']

    # Calculate squared errors
    df_results['Mean Squared Error Rerank'] = df_results['Diff from Rerank'] ** 2
    df_results['Mean Squared Error IAAF'] = df_results['Diff from IAAF'] ** 2

    # Calculate average of non-NaN mean squared errors
    avg_mse_rerank = df_results['Mean Squared Error Rerank'].mean()
    avg_mse_iaaf = df_results['Mean Squared Error IAAF'].mean()

    # Replace NaN values with the average mean squared error
    df_results['Mean Squared Error Rerank'] = df_results['Mean Squared Error Rerank'].fillna(avg_mse_rerank)
    df_results['Mean Squared Error IAAF'] = df_results['Mean Squared Error IAAF'].fillna(avg_mse_iaaf)

    # Recalculate differences based on the square root of mean squared errors
    df_results.loc[df_results['Diff from Rerank'].isna(), 'Diff from Rerank'] = np.sqrt(avg_mse_rerank)
    df_results.loc[df_results['Diff from IAAF'].isna(), 'Diff from IAAF'] = np.sqrt(avg_mse_iaaf)

    # Calculate root mean squared errors
    rmse_rerank = np.sqrt(df_results['Mean Squared Error Rerank'].mean())
    rmse_iaaf = np.sqrt(df_results['Mean Squared Error IAAF'].mean())

    print("Root Mean Squared Error (Rerank):", rmse_rerank)
    print("Root Mean Squared Error (IAAF):", rmse_iaaf)

    return df_results

In [71]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/women/1500-metres/final/result", "1500m", ["https://worldathletics.org/world-rankings/1500m/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/1500m/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"] )

Root Mean Squared Error (Rerank): 36.426867373776005
Root Mean Squared Error (IAAF): 19.284944669588526


<ipython-input-70-831e7de5bebe>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '36.426867373776005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_results.loc[df_results['Diff from Rerank'].isna(), 'Diff from Rerank'] = np.sqrt(avg_mse_rerank)


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,FAITH KIPYEGON,KEN,3:51.29 OR,1500m,1,1.0,0.0,0.000000,0,0.000000
1,2,JESSICA HULL,AUS,3:52.56,1500m,2,3.0,0.0,1.000000,0,1.000000
2,3,GEORGIA BELL,GBR,3:52.61 NR,1500m,7,13.0,4.0,10.000000,16,100.000000
3,4,DIRIBE WELTEJI,ETH,3:52.75 PB,1500m,40,2.0,36.0,-2.000000,1296,4.000000
4,5,LAURA MUIR,GBR,3:53.37 PB,1500m,5,4.0,0.0,-1.000000,0,1.000000
5,6,SUSAN LOKAYO EJORE,KEN,3:56.07 PB,1500m,18,20.0,12.0,14.000000,144,196.000000
6,7,NIKKI HILTZ,USA,3:56.38,1500m,52,19.0,45.0,12.000000,2025,144.000000
7,8,ELLE ST. PIERRE,USA,3:57.52,1500m,16,26.0,8.0,18.000000,64,324.000000
8,9,AGATHE GUILLEMOT,FRA,3:59.08,1500m,26,23.0,17.0,14.000000,289,196.000000
9,10,KLAUDIA KAZIMIERSKA,POL,4:00.12 PB,1500m,82,48.0,72.0,38.000000,5184,1444.000000


In [72]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/women/3000-metres-steeplechase/final/result", "3000m SC", ["https://worldathletics.org/world-rankings/3000msc/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/3000msc/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"] )

Root Mean Squared Error (Rerank): 84.97646733066749
Root Mean Squared Error (IAAF): 7.162938803849459


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,WINFRED MUTILE YAVI,BRN,8:52.76 OR,3000m SC,NaN,NaN,84.976467,7.162939,7221.0,51.307692
1,2,PERUTH CHEMUTAI,UGA,8:53.34 NR,3000m SC,39.0,4.0,37.000000,2.000000,1369.0,4.000000
2,3,FAITH CHEROTICH,KEN,8:55.15 PB,3000m SC,106.0,3.0,103.000000,0.000000,10609.0,0.000000
3,4,ALICE FINOT,FRA,8:58.67 AR,3000m SC,12.0,5.0,8.000000,1.000000,64.0,1.000000
4,5,SEMBO ALMAYEW,ETH,9:00.83 SB,3000m SC,79.0,9.0,74.000000,4.000000,5476.0,16.000000
5,6,BEATRICE CHEPKOECH,KEN,9:04.24,3000m SC,76.0,2.0,70.000000,-4.000000,4900.0,16.000000
6,7,ELIZABETH BIRD,GBR,9:04.35 NR,3000m SC,27.0,11.0,20.000000,4.000000,400.0,16.000000
7,8,LOMI MULETA,ETH,9:06.07 PB,3000m SC,135.0,13.0,127.000000,5.000000,16129.0,25.000000
8,9,NORAH JERUTO,KAZ,9:08.97 SB,3000m SC,163.0,NaN,154.000000,7.162939,23716.0,51.307692
9,10,LEA MEYER,GER,9:09.59 PB,3000m SC,37.0,22.0,27.000000,12.000000,729.0,144.000000


In [73]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/women/800-metres/final/result", "800m", ["https://worldathletics.org/world-rankings/800m/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/800m/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

<ipython-input-70-831e7de5bebe>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '115.80155439371269' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_results.loc[df_results['Diff from Rerank'].isna(), 'Diff from Rerank'] = np.sqrt(avg_mse_rerank)
<ipython-input-70-831e7de5bebe>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '17.240939649566666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_results.loc[df_results['Diff from IAAF'].isna(), 'Diff from IAAF'] = np.sqrt(avg_mse_iaaf)


Root Mean Squared Error (Rerank): 115.80155439371269
Root Mean Squared Error (IAAF): 17.240939649566666


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,KEELY HODGKINSON,GBR,1:56.72,800m,3,1,2.0,0.0,4,0
1,2,TSIGE DUGUMA,ETH,1:57.15 PB,800m,239,9,237.0,7.0,56169,49
2,3,MARY MORAA,KEN,1:57.42,800m,137,2,134.0,-1.0,17956,1
3,4,SHAFIQUA MALONEY,VIN,1:57.66,800m,119,27,115.0,23.0,13225,529
4,5,RÉNELLE LAMOTE,FRA,1:58.19,800m,13,16,8.0,11.0,64,121
5,6,WORKNESH MESELE,ETH,1:58.28,800m,117,21,111.0,15.0,12321,225
6,7,JULIETTE WHITTAKER,USA,1:58.50,800m,78,45,71.0,38.0,5041,1444
7,8,PRUDENCE SEKGODISO,RSA,1:58.79,800m,58,11,50.0,3.0,2500,9


In [74]:
complete_analysis("https://worldathletics.org/competitions/olympic-games/paris24/results/women/5000-metres/final/result", "5000m", ["https://worldathletics.org/world-rankings/5000m/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/5000m/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

<ipython-input-70-831e7de5bebe>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '85.24926685901762' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_results.loc[df_results['Diff from Rerank'].isna(), 'Diff from Rerank'] = np.sqrt(avg_mse_rerank)
<ipython-input-70-831e7de5bebe>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '20.781602440620407' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_results.loc[df_results['Diff from IAAF'].isna(), 'Diff from IAAF'] = np.sqrt(avg_mse_iaaf)


Root Mean Squared Error (Rerank): 85.24926685901762
Root Mean Squared Error (IAAF): 20.781602440620407


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1,BEATRICE CHEBET,KEN,14:28.56,5000m,6,1,5.0,0.0,25,0
1,2,FAITH KIPYEGON,KEN,14:29.60 SB,5000m,1,12,-1.0,10.0,1,100
2,3,SIFAN HASSAN,NED,14:30.61 SB,5000m,90,7,87.0,4.0,7569,16
3,4,NADIA BATTOCLETTI,ITA,14:31.64 NR,5000m,35,13,31.0,9.0,961,81
4,5,MARGARET CHELIMO KIPKEMBOI,KEN,14:32.23 SB,5000m,23,22,18.0,17.0,324,289
5,6,EJGAYEHU TAYE,ETH,14:32.98,5000m,20,3,14.0,-3.0,196,9
6,7,MEDINA EISA,ETH,14:35.43,5000m,115,4,108.0,-3.0,11664,9
7,8,KAROLINE BJERKELI GRØVDAL,NOR,14:43.21,5000m,92,17,84.0,9.0,7056,81
8,9,GUDAF TSEGAY,ETH,14:45.21 SB,5000m,8,2,-1.0,-7.0,1,49
9,10,KARISSA SCHWEIZER,USA,14:45.57,5000m,43,38,33.0,28.0,1089,784


In [75]:
complete_analysis("https://worldathletics.org/en/competitions/olympic-games/paris24/results/women/10000-metres/final/result", "10000m", ["https://worldathletics.org/world-rankings/10000m/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/10000m/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 126.27081286591206
Root Mean Squared Error (IAAF): 17.91381483394204


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1.0,BEATRICE CHEBET,KEN,30:43.25,10000m,6.0,NaN,5.000000,17.913815,25.000000,320.904762
1,2.0,NADIA BATTOCLETTI,ITA,30:43.35 NR,10000m,35.0,14.0,33.000000,12.000000,1089.000000,144.000000
2,3.0,SIFAN HASSAN,NED,30:44.12 SB,10000m,90.0,9.0,87.000000,6.000000,7569.000000,36.000000
3,4.0,MARGARET CHELIMO KIPKEMBOI,KEN,30:44.58,10000m,23.0,22.0,19.000000,18.000000,361.000000,324.000000
4,5.0,LILIAN KASAIT RENGERUK,KEN,30:45.04,10000m,22.0,11.0,17.000000,6.000000,289.000000,36.000000
5,6.0,GUDAF TSEGAY,ETH,30:45.21,10000m,8.0,1.0,2.000000,-5.000000,4.000000,25.000000
6,7.0,FOTYEN TESFAY,ETH,30:46.93,10000m,14.0,16.0,7.000000,9.000000,49.000000,81.000000
7,8.0,WEINI KELATI FREZGHI,USA,30:49.98,10000m,44.0,18.0,36.000000,10.000000,1296.000000,100.000000
8,9.0,KARISSA SCHWEIZER,USA,30:51.99 SB,10000m,43.0,37.0,34.000000,28.000000,1156.000000,784.000000
9,10.0,TSIGIE GEBRESELAMA,ETH,30:54.57,10000m,17.0,5.0,7.000000,-5.000000,49.000000,25.000000


In [76]:
complete_analysis("https://worldathletics.org/en/competitions/olympic-games/paris24/results/women/marathon/final/result", "Marathon", ["https://worldathletics.org/world-rankings/marathon/women?regionType=world&page=1&rankDate=2024-07-23&limitByCountry=0", "https://worldathletics.org/world-rankings/marathon/women?regionType=world&page=2&rankDate=2024-07-23&limitByCountry=0"])

Root Mean Squared Error (Rerank): 258.008270908401
Root Mean Squared Error (IAAF): 83.98605774017996


,Place,Competitor,Nat,Mark,Discipline,Index in rerank,Index in IAAF rank,Diff from Rerank,Diff from IAAF,Mean Squared Error Rerank,Mean Squared Error IAAF
0,1.0,SIFAN HASSAN,NED,2:22:55 OR,Marathon,90.0,2.0,89.000000,1.000000,7921.000000,1.000000
1,2.0,TIGST ASSEFA,ETH,2:22:58,Marathon,70.0,1.0,68.000000,-1.000000,4624.000000,1.000000
2,3.0,HELLEN OBIRI,KEN,2:23:10 PB,Marathon,345.0,11.0,342.000000,8.000000,116964.000000,64.000000
3,4.0,SHARON LOKEDI,KEN,2:23:14 PB,Marathon,NaN,40.0,258.008271,36.000000,66568.267857,1296.000000
4,5.0,AMANE BERISO SHANKULE,ETH,2:23:57,Marathon,200.0,3.0,195.000000,-2.000000,38025.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...
86,NaN,MELAT YISAK KEJETA,GER,DNF,Marathon,218.0,122.0,258.008271,83.986058,66568.267857,7053.657895
87,NaN,JACKLINE SAKILU,TAN,DNF,Marathon,238.0,66.0,258.008271,83.986058,66568.267857,7053.657895
88,NaN,JOAN CHELIMO MELLY,ROU,DNF,Marathon,194.0,145.0,258.008271,83.986058,66568.267857,7053.657895
89,NaN,SINEAD DIVER,AUS,DNF,Marathon,NaN,NaN,258.008271,83.986058,66568.267857,7053.657895


In [77]:
import pandas as pd

data = {
    'WOMEN Event': ['800m', '1500m', '3000m SC', '5000m', '10000m', 'Marathon'],
    'MSE Exponential Decay Rerank (WOMEN)': [115.80155439371269, 36.426867373776005, 84.97646733066749, 85.24926685901762 , 126.27081286591206, 258.008270908401],
    'MSE IAAF Rank (WOMEN)': [17.240939649566666, 19.284944669588526, 7.162938803849459, 20.781602440620407, 17.91381483394204, 83.98605774017996]
}

RMSEdf = pd.DataFrame(data)
RMSEdf


,WOMEN Event,MSE Exponential Decay Rerank (WOMEN),MSE IAAF Rank (WOMEN)
0,800m,115.801554,17.240940
1,1500m,36.426867,19.284945
2,3000m SC,84.976467,7.162939
3,5000m,85.249267,20.781602
4,10000m,126.270813,17.913815
5,Marathon,258.008271,83.986058


Comparing between the men's and womens one below:

In [78]:
import pandas as pd

dataMEN = {
    'Event': ['800m', '1500m', '3000m SC', '5000m', '10000m', 'Marathon'],
    'MSE Exponential Decay Rerank': [52.15361924162119, 49.83556293785928, 107.44145692732702107, 167.09920064782733, 142.40984943953598, 292.56994965067156],
    'MSE IAAF Rank': [41.11265012134343, 13.231905884389192, 14.973309587395834, 30.49672113523026, 33.69050311289518, 86.8996731379892]
}

RMSEdfMEN = pd.DataFrame(dataMEN)
RMSEdfMEN


,Event,MSE Exponential Decay Rerank,MSE IAAF Rank
0,800m,52.153619,41.112650
1,1500m,49.835563,13.231906
2,3000m SC,107.441457,14.973310
3,5000m,167.099201,30.496721
4,10000m,142.409849,33.690503
5,Marathon,292.569950,86.899673


In [79]:
import pandas as pd

RMSEdf.rename(columns={'WOMEN Event': 'Event'}, inplace=True)


combined_RMSEdf = pd.merge(RMSEdf, RMSEdfMEN, on='Event', how='outer')


combined_RMSEdf


,Event,MSE Exponential Decay Rerank (WOMEN),MSE IAAF Rank (WOMEN),MSE Exponential Decay Rerank,MSE IAAF Rank
0,800m,115.801554,17.240940,52.153619,41.112650
1,1500m,36.426867,19.284945,49.835563,13.231906
2,3000m SC,84.976467,7.162939,107.441457,14.973310
3,5000m,85.249267,20.781602,167.099201,30.496721
4,10000m,126.270813,17.913815,142.409849,33.690503
5,Marathon,258.008271,83.986058,292.569950,86.899673


# Claude Config and setup

I wanted to try out Claude 3.5 sonnet just to see how code would be generated and generally (given that it was zero shot) it seems rather accurate! Might just modify the code since it's rather accurate.

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from google.colab import files
import io
import time
import requests
import math



In [ ]:

# Set up Selenium WebDriver for Google Colab
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,424 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,555 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,041 kB]
Hit:14 https:/

In [ ]:

# Step 1: Get all athlete performances >1100 points for 800m -> Marathon
def get_performances(event):
    url = f"https://www.worldathletics.org/records/all-time-toplists/{event}/outdoor/women/senior"
    driver.get(url)

    # Wait for the table to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "records-table")))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', class_='records-table')

    performances = []
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all('td')
        if len(cols) >= 5:
            performance = cols[2].text.strip()
            points = int(cols[4].text.strip())
            if points > 1100:
                performances.append((performance, points))
            else:
                break  # Stop when points fall below 1100

    return performances

events = ['800m', '1500m', '5000m', '10000m', 'marathon']
all_performances = {}

for event in events:
    all_performances[event] = get_performances(event)

# Step 2: Get all outdoor performances and filter out indoor performances
df1 = pd.DataFrame([(event, perf[0], perf[1]) for event, perfs in all_performances.items() for perf in perfs],
                   columns=['Event', 'Performance', 'Points'])

# Step 3: Get list of female athletes who competed in the events through Selenium
def get_athletes(event):
    url = f"https://www.worldathletics.org/records/all-time-toplists/{event}/outdoor/women/senior"
    driver.get(url)

    # Wait for the table to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "records-table")))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', class_='records-table')

    athletes = []
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all('td')
        if len(cols) >= 3:
            athlete = cols[1].text.strip()
            athletes.append(athlete)

    return athletes

all_athletes = []
for event in events:
    all_athletes.extend(get_athletes(event))

df2 = pd.DataFrame(all_athletes, columns=['Athlete'])
df2 = df2.drop_duplicates()

# Step 4: Process names and combine DF1 and DF2
# Assuming the athlete names are in the same format in both dataframes
df1['Athlete'] = df1['Performance'].str.split(' ', n=1).str[1]
df = pd.merge(df1, df2, on='Athlete', how='inner')

print(df.head())
print(f"Total number of performances: {len(df)}")

I think with better prompting Claude's potential is great! but unfortunately needs more customisation, should probably have added some examples.